In [1]:
import pandas as pd
import json
import requests
from collections import defaultdict
from time import sleep
from tqdm import tqdm_notebook as tqdm

# Define Functions

Нужно переопределить директории для файлов под свой комп. Самое важное - под каждый город отдельная папка с названием таким же, как называются листы в Excel'e с гридами


Общий вид пути:

```{python}
'D:/папка где у тебя проекты/папка проекта/папка с данными/папка города/сегодняшняя дата.csv'
```

In [2]:
def send_request(bbox):    
    url = 'https://www.cian.ru/cian-api/mobile-site/v2/offers/clusters/'

    params = {
        'deal_type' : 'rent', 'engine_version' : 2,
        'offer_type' : 'offices', 'office_type[0]' : 1,
        'office_type[10]' : 12, 'office_type[1]' : 2,
        'office_type[2]' : 3, 'office_type[3]' : 4,
        'office_type[4]' : 5, 'office_type[5]' : 6,
        'office_type[6]' : 7, 'office_type[7]' : 9,
        'office_type[8]' : 10, 'office_type[9]' : 11,
        'screen_area': 354,
        'bbox' : f"{bbox['minx']},{bbox['miny']},{bbox['maxx']},{bbox['maxy']}",
        'deal_type' : 1, 'allow_precision_correction' : 0, 'zoom' : 15       
    }

    data = requests.get(url, params = params).json()
    
    return data

In [3]:
def get_data(name, grid):
    datums = defaultdict(dict)

    for i, bbox in tqdm(grid.iterrows(), total = grid.shape[0], desc = name):
#         pass
        try:
            datum = send_request(bbox)
            datums[i] = datum
        except Exception as e:
            try:
                if e[0] == 'Connection aborted.':
                    sleep(2)
                    datum = send_request(bbox)
                    datums[i] = datum
            except:
                continue
        sleep(0.5)
        if i > 0 and i % 1000 == 0:
            with open('D:/data_projects/commerical_death/datum_cian_{}.json'.format(name), 'w') as outfile:
                json.dump(datums, outfile)

    with open('D:/data_projects/commerical_death/datum_cian_{}.json'.format(name), 'w') as outfile:
        json.dump(datums, outfile)

In [4]:
def parce_json(name, today):
    with open('D:/data_projects/commerical_death/datum_cian_{}.json'.format(name)) as infile:
        datums = json.load(infile)

    series_list = []
    for key, datum in datums.items():
        for offer in datum['data']['offers']:
            series_list.append(pd.Series(offer))

#     cian_data = pd.DataFrame(series_list).set_index('cian_id')
    cian_data = pd.DataFrame(series_list)
    
    cian_data.to_csv('D:/data_projects/commerical_death/{}/{}.csv'.format(name, today), index = None)


# What day is it today?

In [5]:
today = str(pd.datetime.today().date())

print(today)

2020-07-04


In [2]:
today = '2020-06-27'
today

'2020-06-27'

# Getting data

## City selector pt.1a

In [7]:
my_filter = ['Moscow', 'NizNovgorod', 'Kazan']


grids = pd.read_excel('City_Grids.xlsx', sheet_name = my_filter)

## City selector pt.1b

In [13]:
my_filter = ['Ufa', 'Ekaterinburg', 'Tyumen', 'Omsk']


grids = pd.read_excel('City_Grids.xlsx', sheet_name = my_filter)

## City selector pt.2

In [8]:
andrew_filter = ['Novosibirsk', 'Krasnoyarsk', 'Irkutsk', 'Chita',
                 'Blagoveschensk', 'Khabarovsk', 'Vladivostok']

grids = pd.read_excel('City_Grids.xlsx', sheet_name = andrew_filter)

## Getting data, storing to files

In [8]:
# today = str(pd.datetime.today().date())
for name, grid in grids.items():
    get_data(name, grid)
    parce_json(name, today)